In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install onnx

Cloning into 'yolov5'...
remote: Enumerating objects: 12936, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 12936 (delta 201), reused 278 (delta 168), pack-reused 12618
Receiving objects: 100% (12936/12936), 12.63 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (8852/8852), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 31.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 22.6 MB/s 


In [3]:
!pip install torch==1.11 torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 750.6 MB 12 kB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.

In [27]:
!python export.py --weights yolov5s.pt --include onnx --simplify

export: data=data/coco128.yaml, weights=['yolov5s.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v6.1-394-gd7bc5d7 Python-3.7.13 torch-1.11.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients

PyTorch: starting from yolov5s.pt with output shape (1, 25200, 85) (14.1 MB)

ONNX: starting export with onnx 1.12.0...
ONNX: simplifying with onnx-simplifier 0.4.7...
ONNX: export success, saved as yolov5s.onnx (28.0 MB)

Export complete (3.78s)
Results saved to /content/yolov5
Detect:          python detect.py --weights yolov5s.onnx 
Validate:        python val.py --weights yolov5s.onnx 
PyTorch Hub:     model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5s.onnx')
Visualize:

In [28]:
import cv2
import numpy as np


def inference(input_image, net):
    # Create a 4D blob from a frame
    blob = cv2.dnn.blobFromImage(input_image, 1/255., mean=[0, 0, 0], swapRB=1, crop=False)

    # Set the input to the network
    net.setInput(blob)

    # Run the forward pass to get output of the output layers
    # outputs = net.forward(net.getUnconnectedOutLayersNames())
    outputs = net.forward()

    return outputs


if __name__ == '__main__':
    # Load an image
    image = cv2.imread('zidane.jpg')
    image = cv2.resize(image, (640, 640))

    # load the network
    net = cv2.dnn.readNet('yolov5s.onnx')

    # Do inference
    outputs = inference(image, net)
    print(outputs)

[[[3.84279442e+00 3.84001207e+00 8.80481625e+00 ... 1.99058396e-03
   1.19962380e-03 4.73506562e-03]
  [1.34774103e+01 4.43537617e+00 2.78068943e+01 ... 2.24056421e-03
   1.91699003e-03 4.81091440e-03]
  [1.79742661e+01 4.59590435e+00 3.27912827e+01 ... 2.23641819e-03
   1.47986808e-03 4.59005823e-03]
  ...
  [5.65978882e+02 6.13254028e+02 1.39626022e+02 ... 2.05551041e-03
   1.16600911e-03 8.56585975e-04]
  [5.92219604e+02 6.06922180e+02 1.06126694e+02 ... 1.79403019e-03
   9.45722219e-04 8.32671125e-04]
  [6.17259888e+02 6.16553284e+02 1.19196045e+02 ... 1.82165217e-03
   9.16930672e-04 8.63867463e-04]]]
